In [1]:
%load_ext tensorboard
import datetime
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

import os 
import cv2
import urllib
import requests
import PIL.Image
import numpy as np
from bs4 import BeautifulSoup

In [22]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout


# Define the AlexNet Model 
class AlexNet(Sequential):
    def __init__(self, input_shape, num_classes):
        super().__init__()

        # No.1 Convolutional Layer: (227- 11 + 2 * 0) / 4 + 1 = 55
        self.add(Conv2D(filters=96, kernel_size=(11,11), strides=4,
                        padding='valid', activation='relu',
                        input_shape=input_shape, kernel_initializer='GlorotNormal'))
        # Max Pooling: (55- 3 + 2 * 0) / 2 + 1 = 27
        self.add(MaxPooling2D(pool_size=(3,3), strides=(2,2),
                              padding='valid', data_format=None))
        # Add Dropout
        #self.add(Dropout(0.05))
        
        # No.2 Conv Layer: (27- 5 + 2 * 2) / 1 + 1 = 27
        self.add(Conv2D(filters=256, kernel_size=(5,5), strides=1,
                        padding='same', activation='relu',
                        kernel_initializer='GlorotNormal'))
        # Max Pooling: (27-  3 + 2 * 0) / 2 + 1 = 13
        self.add(MaxPooling2D(pool_size=(3,3), strides=(2,2),
                              padding='valid', data_format=None))
        
        # No.3 Conv Layer: (13 - 3 + 2 * 1) / 1 + 1 = 13
        self.add(Conv2D(filters=384, kernel_size=(3,3), strides=1,
                        padding='same', activation='relu',
                        kernel_initializer='GlorotNormal'))

        # No.4 Conv Layer: (13 - 3 + 2 * 1) / 1 + 1 = 13
        self.add(Conv2D(filters=384, kernel_size=(3,3), strides=1,
                 padding='same', activation='relu',
                 kernel_initializer='GlorotNormal'))

        # No.5 Conv Layer: (13 - 3 + 2 * 1) / 1 + 1 = 13
        self.add(Conv2D(filters=256, kernel_size=(3,3), strides=1,
                 padding='same', activation='relu',
                 kernel_initializer='GlorotNormal')) 
        # Max Pooling: (13 - 3 + 2 * 0) / 2 + 1 =  6
        self.add(MaxPooling2D(pool_size=(3,3), strides=(2,2),
                              padding='valid', data_format=None))

        # # Flatten the three dimensions of 256 x 6 x 6 into one dimension of 9216.
        self.add(Flatten())

        # No.6 FC Layer 
        self.add(Dense(4096, activation='relu'))
        # Add Dropout
        #self.add(Dropout(0.2))
        
        # No.7 FC Layer         
        self.add(Dense(4096, activation='relu'))
        # Add Dropout 
        #self.add(Dropout(0.2))

        # No.8 FC Layer
        self.add(Dense(1000, activation='relu'))
        self.add(Dense(num_classes, activation='softmax'))
        

In [3]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation,BatchNormalization, Flatten, Conv2D, MaxPooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import os

In [4]:
BATCH_SIZE = 32
image_width = 227
image_height = 227
channels = 3
num_classes = 135

model = AlexNet((image_width,image_height,channels), num_classes)

In [23]:
opt = tf.optimizers.Adagrad(learning_rate=0.00495, epsilon=None, decay=0.0)
model.compile(optimizer = opt , loss = tf.keras.losses.KLDivergence() , metrics=['accuracy'])

In [14]:
train_path = r"C:\Users\GUITA\Desktop\Stuff\Projects\Plant CLEF\alexnet\dataset\seg_train\seg_train"
test_path = r"C:\Users\GUITA\Desktop\Stuff\Projects\Plant CLEF\alexnet\dataset\seg_test\seg_test"

In [9]:
model.summary()

Model: "alex_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 55, 55, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 256)       614656    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 256)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 13, 13, 384)       885120    
                                                                 
 conv2d_3 (Conv2D)           (None, 13, 13, 384)       132

In [19]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=10,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.1,
                                   zoom_range=0.1)

train_generator = train_datagen.flow_from_directory(train_path,
                                                    target_size=(image_height, image_width),
                                                    color_mode="rgb",
                                                    batch_size=BATCH_SIZE,
                                                    seed=1,
                                                    shuffle=True,
                                                    class_mode="categorical")

valid_datagen = ImageDataGenerator(rescale=1.0/255.0)

valid_generator = valid_datagen.flow_from_directory(test_path,
                                                    target_size=(image_height, image_width),
                                                    color_mode="rgb",
                                                    batch_size=BATCH_SIZE,
                                                    seed=7,
                                                    shuffle=True,
                                                    class_mode="categorical")

# The terminal shows the number of images belonging to 2 classes. 
train_num = train_generator.samples
valid_num = valid_generator.samples

Found 6259 images belonging to 135 classes.
Found 6042 images belonging to 135 classes.


In [20]:
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
callback_list = [tensorboard_callback]

In [24]:
model.fit(train_generator, epochs=15)

Epoch 1/15
133/196 [===================>..........] - ETA: 1:30 - loss: 4.4985 - accuracy: 0.0146

KeyboardInterrupt: 